In [2]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from pathlib import Path

# Load environment variables from .env file
current_dir = os.getcwd()
env_path = os.path.join(current_dir, '.env')
# Access your API key from the .env file
load_dotenv(dotenv_path=env_path)  
api_key = os.getenv("API_KEY")
# Initialize the model with the API key
model = ChatOpenAI(model="llama-3.3-70b-versatile", api_key=api_key, base_url="https://api.groq.com/openai/v1")

我們也可以自己寫agent: https://langchain-ai.github.io/langgraph/how-tos/react-agent-from-scratch/

langgraph swarm: https://langchain-ai.github.io/langgraph/reference/swarm/

`config` object in LangGraph Swarm, specifically the {"configurable": {"thread_id": "1"}} structure, is not modified by the system during execution. It serves as a persistent identifier that you provide to maintain conversation state across multiple interactions

`router` in `StateGraph`: router refers to a special type of node or function that determines the flow of execution through the graph based on the current state. It acts as a decision-making component that directs which node should be executed next.

In [3]:
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.store.memory import InMemoryStore
from langgraph.prebuilt import create_react_agent
from langgraph_swarm import create_handoff_tool, create_swarm

In [ ]:


def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b

alice = create_react_agent(
    model,
    [add, create_handoff_tool(agent_name="Bob")],
    prompt="You are Alice, an addition expert.",
    name="Alice",
)

bob = create_react_agent(
    model,
    [create_handoff_tool(agent_name="Alice", description="Transfer to Alice, she can help with math")],
    prompt="You are Bob, you speak like a pirate.",
    name="Bob",
)
# short-term memory
# maintain conversation state across interactions
checkpointer = InMemorySaver()
# long-term memory
store = InMemoryStore()

workflow = create_swarm(
    [alice, bob],
    default_active_agent="Alice"
)

# Compiles the state graph into a CompiledStateGraph object.
app = workflow.compile(
    checkpointer=checkpointer,
    store=store
)
config = {"configurable": {"thread_id": "1"}}
turn_1 = app.invoke(
    {"messages": [{"role": "user", "content": "i'd like to speak to Bob"}]},
    config,
)
print(turn_1)
turn_2 = app.invoke(
    {"messages": [{"role": "user", "content": "what's 5 + 7?"}]},
    config,
)
print(turn_2)

{'messages': [HumanMessage(content="i'd like to speak to Bob", additional_kwargs={}, response_metadata={}, id='b35e6d5a-3a62-46c2-8ca8-0a4b945b3b94'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_2966', 'function': {'arguments': '{}', 'name': 'transfer_to_bob'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 275, 'total_tokens': 287, 'completion_tokens_details': None, 'prompt_tokens_details': None, 'queue_time': 0.262570968, 'prompt_time': 0.024186555, 'completion_time': 0.043636364, 'total_time': 0.067822919}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_3f3b593e33', 'id': 'chatcmpl-2324f17b-a548-40ad-a1e7-c760ebfb415d', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, name='Alice', id='run--6bd1fe6e-6074-4935-80da-7399e5f7cf0a-0', tool_calls=[{'name': 'transfer_to_bob', 'args': {}, 'id': 'call_2966', 'type': 'tool_call'}], usage_metadata={'input_t

Try to add orchestrator agent

In [ ]:
orchestrator = create_react_agent(
    model,
    [create_handoff_tool(agent_name="alice"), create_handoff_tool(agent_name="bob")],
    prompt="""You are a meta-expert orchestrator. Your job is to:
    1. Analyze the user's request
    2. Plan which agents should handle different aspects of the task
    3. Create specific instructions for each agent using meta-prompting
    4. Hand off to the appropriate agent
    When all subtasks are complete, hand off to the synthesizer.""",
    name="orchestrator"
)

alice = create_react_agent(
    model,
    [add, create_handoff_tool(agent_name="orchestrator"), 
     create_handoff_tool(agent_name="synthesizer")],
    prompt="You are Alice, an addition expert. Follow the specific instructions provided by the orchestrator.",
    name="alice"
)

bob = create_react_agent(
    model,
    [create_handoff_tool(agent_name="orchestrator"), 
     create_handoff_tool(agent_name="synthesizer")],
    prompt="You are Bob, you speak like a pirate. Follow the specific instructions provided by the orchestrator.",
    name="bob"
)

synthesizer = create_react_agent(
    model,
    [create_handoff_tool(agent_name="orchestrator")],
    prompt="""You are a synthesizer agent. Your job is to:
    1. Review all the work done by other agents
    2. Combine their outputs into a coherent, comprehensive response
    3. Ensure the final answer addresses all aspects of the original query""",
    name="synthesizer"
)



agent dynamically generation 

In [7]:
import os
import json
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.store.memory import InMemoryStore
from langgraph.prebuilt import create_react_agent
from langgraph_swarm import create_handoff_tool, create_swarm

# Load environment variables from .env file
current_dir = os.getcwd()
env_path = os.path.join(current_dir, '.env')
load_dotenv(dotenv_path=env_path)  
api_key = os.getenv("API_KEY")
# Initialize the model with the API key
model = ChatOpenAI(model="llama-3.3-70b-versatile", api_key=api_key, base_url="https://api.groq.com/openai/v1")

# Function to generate personas using direct LLM call
def generate_personas(product_name):
    prompt = f"""Create 3 distinct customer personas who would be interested in purchasing {product_name}.
    For each persona, include:
    - A brief, descriptive name (e.g., "Tech-Savvy Tony," "Budget-Conscious Brenda")
    - Key demographic information (age range, occupation, general lifestyle)
    - Their primary motivations and needs related to this type of product
    - Any potential concerns or hesitations they might have
    
    Format your response as a JSON array of persona objects with these fields:
    - name: The persona's descriptive name
    - demographics: Key demographic information
    - motivations: Primary motivations and needs
    - concerns: Potential concerns or hesitations
    
    Only respond with the JSON array, nothing else."""
    
    response = model.invoke(prompt)
    try:
        # Try to extract JSON from the response
        content = response.content
        # Find JSON in the content if it's not pure JSON
        json_start = content.find('[')
        json_end = content.rfind(']') + 1
        if json_start >= 0 and json_end > json_start:
            json_str = content[json_start:json_end]
            return json.loads(json_str)
        else:
            # Fallback to default personas
            return default_personas()
    except (json.JSONDecodeError, AttributeError):
        return default_personas()

# Default personas in case of parsing issues
def default_personas():
    return [
        {
            "name": "Tech-Savvy Tony",
            "demographics": "28-35, software engineer, urban lifestyle",
            "motivations": "Cutting-edge features, seamless integration, innovation",
            "concerns": "Privacy, update frequency, obsolescence"
        },
        {
            "name": "Budget-Conscious Brenda",
            "demographics": "35-45, mid-level manager, suburban parent",
            "motivations": "Value for money, durability, practical features",
            "concerns": "Price, warranty, maintenance costs"
        },
        {
            "name": "Eco-Friendly Ethan",
            "demographics": "25-40, environmental consultant, mixed urban-rural lifestyle",
            "motivations": "Sustainable materials, energy efficiency, environmental ethics",
            "concerns": "Carbon footprint, recyclability, greenwashing"
        }
    ]

# Function to create agents dynamically based on persona data
def create_dynamic_agents(model, persona_data):
    agents = []
    agent_names = []
    
    for persona in persona_data:
        agent_name = persona["name"].lower().replace(" ", "_").replace("-", "_")
        agent_names.append(agent_name)
        
        prompt = f"""You are {persona['name']}, with the following characteristics:

Demographics: {persona['demographics']}

Motivations: {persona['motivations']}

Concerns: {persona['concerns']}

When discussing products, you should reflect these characteristics in your perspective, questions, and statements. Always stay in character and express views consistent with your persona's background and priorities."""
        
        agent = create_react_agent(
            model,
            [create_handoff_tool(agent_name="discussion_coordinator")],
            prompt=prompt,
            name=agent_name
        )
        agents.append(agent)
    
    return agents, agent_names

# Main function to run the workflow
def run_persona_discussion(product_name):
    # Generate personas using direct LLM call
    personas = generate_personas(product_name)
    print(f"Generated {len(personas)} personas for {product_name}")
    
    # Create dynamic agents based on personas and get their names
    dynamic_agents, agent_names = create_dynamic_agents(model, personas)
    print(f"Created {len(dynamic_agents)} dynamic agents: {', '.join(agent_names)}")
    
    # Create handoff tools for the discussion coordinator
    coordinator_tools = [create_handoff_tool(agent_name="synthesizer")]
    for agent_name in agent_names:
        coordinator_tools.append(create_handoff_tool(agent_name=agent_name))
    
    # Discussion Coordinator with handoff tools to all agents
    discussion_coordinator = create_react_agent(
        model,
        coordinator_tools,
        prompt=f"""You are a Discussion Coordinator. Your task is to:
            1. Facilitate a discussion between persona agents about {product_name}
            2. Hand off to each agent to get their perspective
            3. After all agents have contributed, hand off to the Synthesizer
            
            You can hand off to the following agents:
            {', '.join(agent_names)}
            
            Start by introducing the product and handing off to the first agent to get their perspective.
            After each agent responds, hand off to another agent until all have contributed.
            Then hand off to the synthesizer to summarize the discussion.
            """,
        name="discussion_coordinator"
    )
    
    # Synthesizer
    synthesizer = create_react_agent(
        model,
        [],
        prompt=f"""You are a Synthesizer. Your task is to:
            1. Review the discussion between the persona agents about {product_name}
            2. Provide a concise summary of the main points discussed
            3. Highlight the different perspectives of the personas
            4. Identify any potential overall conclusions or areas of interest regarding the product
            
            Your summary should be well-structured and insightful, capturing the essence of each persona's concerns and interests.
            """,
        name="synthesizer"
    )
    
    # Set up storage
    checkpointer = InMemorySaver()
    store = InMemoryStore()
    
    # Create workflow with all agents
    all_agents = [discussion_coordinator, synthesizer] + dynamic_agents
    workflow = create_swarm(
        all_agents,
        default_active_agent="discussion_coordinator"
    )
    
    # Compile the workflow
    app = workflow.compile(
        checkpointer=checkpointer,
        store=store
    )
    
    # Initialize the conversation
    config = {"configurable": {"thread_id": "product_discussion"}}
    current_result = app.invoke(
        {"messages": [{"role": "user", "content": f"Facilitate a discussion about {product_name} between the persona agents, then synthesize their perspectives."}]},
        config
    )
    
    # Track which agents have spoken to ensure all participate
    agents_spoken = set()
    max_turns = 15  # Safety limit to prevent infinite loops
    turn_count = 0
    full_conversation = []
    
    # Save the first result
    full_conversation.append(current_result)
    
    # Continue the conversation until all agents have spoken and the synthesizer concludes
    while turn_count < max_turns:
        turn_count += 1
        print(f"Turn {turn_count}: Active agent is {current_result.get('active_agent', 'unknown')}")
        
        # Print the latest message from the current active agent
        if "messages" in current_result and current_result["messages"]:
            latest_message = current_result["messages"][-1]
            try:
                if isinstance(latest_message, dict):
                    if latest_message.get("role") == "assistant":
                        print(f"{active_agent}: {latest_message['content']}\n")
                elif hasattr(latest_message, "type") and latest_message.type == "ai":
                    print(f"{active_agent}: {latest_message.content}\n")
            except Exception as e:
                print(f"Could not print message: {e}")
        
        # Extract the active agent from the result
        active_agent = current_result.get("active_agent", "")
        
        # If we've reached the synthesizer, we're done after this turn
        if active_agent == "synthesizer":
            # One more invoke to get the synthesizer's response
            next_input = {"messages": current_result["messages"]}
            current_result = app.invoke(next_input, config)
            full_conversation.append(current_result)
            break
            
        # Add the active agent to our tracking set if it's a persona agent
        if active_agent not in ["discussion_coordinator", "synthesizer"]:
            agents_spoken.add(active_agent)
            
        # If all agents have spoken and we're back to the coordinator, prompt to wrap up
        if len(agents_spoken) == len(agent_names) and active_agent == "discussion_coordinator":
            next_input = {"messages": current_result["messages"] + [
                {"role": "user", "content": "All agents have shared their perspectives. Please hand off to the synthesizer to summarize the discussion."}
            ]}
        else:
            # Otherwise, just continue the conversation
            next_input = {"messages": current_result["messages"]}
            
        # Invoke the next turn
        current_result = app.invoke(next_input, config)
        full_conversation.append(current_result)
    
    # Combine all results into a single result with the complete conversation
    final_result = current_result
    
    # Print a summary of the conversation
    print(f"\nDiscussion completed in {turn_count+1} turns")
    print(f"Agents who participated: {', '.join(agents_spoken)}")
    
    return final_result, personas


# Example usage
if __name__ == "__main__":
    product_name = "Smart Home Energy Management System"
    result, personas = run_persona_discussion(product_name)
    
    # Print the personas
    print("\n=== PERSONAS ===\n")
    for i, persona in enumerate(personas, 1):
        print(f"Persona {i}: {persona['name']}")
        print(f"Demographics: {persona['demographics']}")
        print(f"Motivations: {persona['motivations']}")
        print(f"Concerns: {persona['concerns']}")
        print()
    
    # Print the final result
    print("\n=== DISCUSSION RESULT ===\n")
    for message in result["messages"]:
        if hasattr(message, "type") and message.type == "ai":  # or message.type == "assistant"
            print(f"{message.content}\n")


Generated 3 personas for Smart Home Energy Management System
Created 3 dynamic agents: eco_friendly_emma, tech_savvy_tom, budget_conscious_brian
Turn 1: Active agent is eco_friendly_emma
Could not print message: cannot access local variable 'active_agent' where it is not associated with a value
Turn 2: Active agent is eco_friendly_emma
eco_friendly_emma: 

Turn 3: Active agent is tech_savvy_tom
eco_friendly_emma: As Tech-Savvy Tom, I'm excited to share my thoughts on Smart Home Energy Management Systems. My primary motivation is to streamline home automation and energy management, so I'm looking for a system that can integrate with my existing smart devices and provide advanced features like remote monitoring and control.

I'm also concerned about system security and data privacy, as I believe that's crucial for any smart home system. I'd like to hear more about the security features of different Smart Home Energy Management Systems and how they can protect my personal data.

From my p

Interactive agents

In [11]:
def format_agent_response(response):
    """Format the agent response for better readability."""
    agent_name = response.get("active_agent", "Unknown")
    messages = response.get("messages", [])

    if not messages:
        return f"[{agent_name}] No response."

    last_message = messages[-1]
    content = getattr(last_message, 'content', "")  # Access 'content' attribute directly

    return f"[{agent_name}] {content}"

def interactive_chat():
    """Run an interactive chat session with the agent swarm"""
    print("Welcome to the Agent Swarm Chat!")
    print("Type 'exit' or 'quit' to end the conversation.")
    print("Starting with Alice as the default agent.")
    print("-" * 50)
    
    while True:
        # Get user input
        user_input = input("You: ")
        
        # Check if user wants to exit
        if user_input.lower() in ["exit", "quit"]:
            print("Goodbye!")
            break
        
        # Create a message with just the current user input
        # The checkpointer will maintain the conversation state
        current_message = [{"role": "user", "content": user_input}]
        
        # Invoke the agent swarm with just the new message
        response = app.invoke(
            {"messages": current_message},
            config,
        )
        
        # Extract and display the agent's response
        formatted_response = format_agent_response(response)
        print(formatted_response)


In [12]:
interactive_chat()

Welcome to the Agent Swarm Chat!
Type 'exit' or 'quit' to end the conversation.
Starting with Alice as the default agent.
--------------------------------------------------
[Unknown] [
{
"name": "Sweet-Toothed Sally",
"demographics": "18-30, student or young professional, active social life",
"motivations": "Craves unique and exciting flavors, values convenience and affordability",
"concerns": "Calorie intake, potential allergens or sensitivities, limited dietary options"
},
{
"name": "Family-Focused Frank",
"demographics": "30-50, parent or guardian, suburban family lifestyle",
"motivations": "Wants to provide treats for family members, values variety packs and bulk options",
"concerns": "Added sugars, artificial ingredients, and potential choking hazards for young children"
},
{
"name": "Nostalgic Nancy",
"demographics": "40-60, established career, nostalgic for childhood treats",
"motivations": "Seeks classic candy flavors and textures, values sentimental value and retro packaging",